# Null Geodesic Nedir?

* Genel görelilikte **geodesic** bir parçacığın (veya ışığın) uzay-zamandaki izlediği en kısa/uzun yol gibi düşünebilirsin.
* Eğer geodesic **null (ışık benzeri)** ise bu şu demek:

$$ds^2 = 0$$

yani parçacığın **ışık hızında** gittiği durumda, uzay-zaman aralığı sıfırdır.

## Kerr Metriğinde Null Geodesic

Kerr metriğinde bu şu anlama geliyor:
* Kütle çekim alanında ışık ışınlarının yolu, hem **kütle çekimi** hem de **dönüş momenti (a)** yüzünden bükülür.
* Dönüş (frame dragging) ışığı kendi etrafında sürükler.

# Kerr Metriği İçin Null Geodesic Denklemleri

Bunlar genellikle **Hamilton-Jacobi** formundan türetilir:

$$\Sigma \frac{dr}{d\lambda} = \pm \sqrt{R(r)}$$

$$\Sigma \frac{d\theta}{d\lambda} = \pm \sqrt{\Theta(\theta)}$$

$$\Sigma \frac{d\phi}{d\lambda} = \frac{a(E(r^2+a^2)-aL_z)}{\Delta} + \frac{L_z}{\sin^2\theta} - aE$$

$$\Sigma \frac{dt}{d\lambda} = \frac{(r^2+a^2)^2-a^2\Delta\sin^2\theta}{\Delta} E - a(aE\sin^2\theta - L_z)$$

## Parametreler

Burada:
* $E$ → enerjinin birim kütleye oranı (ışık için normalleştirilmiş)
* $L_z$ → açısal momentum
* $a$ → Kerr dönüş parametresi
* $\Delta = r^2 - 2Mr + a^2$
* $\Sigma = r^2 + a^2 \cos^2\theta$

In [1]:
!pip install numpy
!pip install matplotlib
!pip install scipy

In [ ]:
# Colab / Jupyter hücresi: Kerr null geodesics + 3D animasyon
# Gereksinimler: numpy, scipy, matplotlib
import numpy as np
from math import sin, cos
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp

# ----- Kerr parametreleri -----
M = 1.0             # kütle
a = 0.7             # spin parametresi (|a| < M ideal)
r_plus  = M + np.sqrt(max(0.0, M**2 - a**2))
r_minus = M - np.sqrt(max(0.0, M**2 - a**2))

# ----- Yardımcı fonksiyonlar (Carter ayrılabilirliği) -----
def Delta(r): return r*r - 2*M*r + a*a
def Sigma(r, th): return r*r + (a*cos(th))**2

# R(r) ve Theta(θ) fonksiyonları (null geodesic)
# E: enerji (foton için ölçeklendirilebilir, biz E=1 alacağız)
def R_of_r(r, E, Lz, Q):
    term1 = ( (r*r + a*a)*E - a*Lz )
    return term1*term1 - Delta(r)*( Q + (Lz - a*E)**2 )

def Theta_of_th(th, E, Lz, Q):
    # for null geodesics and E maybe 1; expression general
    return Q - ( (a*a)*(1 - E*E) * (cos(th)**2) ) - ( Lz**2 / (np.sin(th)**2) ) * (cos(th)**2) + 0
    # safer full Carter: Θ(θ) = Q - cos^2θ*( a^2*(1-E^2) + Lz^2/sin^2θ )
    # We'll use full formula to avoid mistakes:
def Theta_of_th(th, E, Lz, Q):
    # avoid division by zero at poles by small offset
    s = np.sin(th)
    if abs(s) < 1e-8:
        s = 1e-8
    return Q - (np.cos(th)**2) * ( a*a * (1.0 - E*E) + (Lz*Lz)/(s*s) )

# dphi/dλ and dt/dλ expressions (Σ * der = ...)
def phi_dot(r, th, E, Lz, Q):
    # Σ * dφ/dλ = a*(E*(r^2+a^2)-a Lz)/Δ + Lz/sin^2θ - a E
    s = np.sin(th)
    if abs(s) < 1e-8: s = 1e-8
    numerator = a*( E*(r*r + a*a) - a*Lz ) / max(Delta(r), 1e-12)
    return ( numerator + Lz/(s*s) - a*E ) / Sigma(r, th)

def t_dot(r, th, E, Lz, Q):
    # Σ * dt/dλ = ((r^2+a^2)^2 - a^2 Δ sin^2θ)/Δ * E - a (a E sin^2θ - Lz)
    s2 = np.sin(th)**2
    top = (r*r + a*a)**2 - a*a * Delta(r) * s2
    return ( top * E / max(Delta(r), 1e-12) - a*( a*E*s2 - Lz ) ) / Sigma(r, th)

# Choose constants for photons
E = 1.0  # set energy scale
# We'll create a function to integrate a photon geodesic for given (Lz, Q) and initial pos
def integrate_geodesic(r0, th0, phi0, E, Lz, Q, r_sign=-1.0, max_affine=5000, r_stop_min=0.99*r_plus, r_stop_max=1e3):
    """
    Integrate geodesic in Boyer-Lindquist coordinates using separation:
    State vector: [r, theta, phi, t]
    We integrate using derivatives dr/dλ = pr / Σ, dθ/dλ = pθ / Σ
    But pr = ± sqrt(R(r)), pθ = ± sqrt(Θ(θ)) where signs chosen by r_sign and th_sign
    To get smooth integration, we will integrate system for r and theta using
    dr/dλ = vr, dθ/dλ = vth and compute phi_dot, t_dot as algebraic.
    """
    # initial signs: choose inbound dr < 0
    sign_r = -1.0 if r_sign < 0 else 1.0

    # define ODE right-hand side: y = [r, th, phi, t]
    def rhs(lam, y):
        r = y[0]; th = y[1]; ph = y[2]; tvar = y[3]
        # ensure theta in (0, pi)
        if th <= 1e-6: th = 1e-6
        if th >= np.pi-1e-6: th = np.pi-1e-6

        Sig = Sigma(r, th)
        Rval = R_of_r(r, E, Lz, Q)
        Thval = Theta_of_th(th, E, Lz, Q)
        # numerical safety
        Rval = max(Rval, 0.0)
        Thval = max(Thval, 0.0)

        dr = sign_r * np.sqrt(Rval) / Sig
        dth = np.sqrt(Thval) / Sig  # pick positive polar motion; could use sign
        dphi = phi_dot(r, th, E, Lz, Q)
        dt = t_dot(r, th, E, Lz, Q)
        return [dr, dth, dphi, dt]

    # initial state
    y0 = [r0, th0, phi0, 0.0]
    # integrate until hitting horizon or escaping far or max_affine
    lam_span = (0.0, max_affine)
    # event functions for stopping:
    def hit_inner(lam, y):
        return y[0] - r_stop_min
    hit_inner.terminal = True
    hit_inner.direction = -1  # trigger when decreasing past r_stop_min

    def escape_far(lam, y):
        return y[0] - r_stop_max
    escape_far.terminal = True
    escape_far.direction = 1

    sol = solve_ivp(rhs, lam_span, y0, method='RK45', max_step=1.0, events=(hit_inner, escape_far), atol=1e-8, rtol=1e-7)
    return sol

# ----- Prepare several geodesics (different impact parameters) -----
geodesics = []
params = [
    # (r0, th0, phi0, Lz, Q)
    (50.0, np.pi/2 * 0.99, 0.0, 2.5, 0.0),  # near-equatorial grazing
    (50.0, np.pi/2 * 0.9, 0.2, 3.0, 1.0),
    (50.0, np.pi/2 * 0.8, 1.0, 4.0, 2.0),
]

for (r0, th0, phi0, Lz, Q) in params:
    sol = integrate_geodesic(r0, th0, phi0, E, Lz, Q, r_sign=-1.0, max_affine=2000, r_stop_min=0.99*r_plus, r_stop_max=500.0)
    geodesics.append((sol, Lz, Q))

# ----- Convert solutions to Cartesian for plotting -----
def bl_to_cart(r_arr, th_arr, ph_arr):
    x = r_arr * np.sin(th_arr) * np.cos(ph_arr)
    y = r_arr * np.sin(th_arr) * np.sin(ph_arr)
    z = r_arr * np.cos(th_arr)
    return x, y, z

traj_xyz = []
for sol, Lz, Q in geodesics:
    r_arr = sol.y[0]
    th_arr = sol.y[1]
    ph_arr = sol.y[2]
    x, y, z = bl_to_cart(r_arr, th_arr, ph_arr)
    traj_xyz.append((x, y, z, sol))

# ----- Build surfaces: ergosphere and horizons (for visualization) -----
theta_s = np.linspace(0, np.pi, 80)
phi_s = np.linspace(0, 2*np.pi, 120)
T, P = np.meshgrid(theta_s, phi_s)
r_erg = M + np.sqrt(np.maximum(0.0, M*M - a*a * np.cos(T)**2))
X_e = r_erg * np.sin(T) * np.cos(P)
Y_e = r_erg * np.sin(T) * np.sin(P)
Z_e = r_erg * np.cos(T)

# horizons radii
r_plus_val = r_plus
r_minus_val = r_minus
X_h_plus = r_plus_val * np.sin(T) * np.cos(P)
Y_h_plus = r_plus_val * np.sin(T) * np.sin(P)
Z_h_plus = r_plus_val * np.cos(T)
X_h_minus = r_minus_val * np.sin(T) * np.cos(P)
Y_h_minus = r_minus_val * np.sin(T) * np.sin(P)
Z_h_minus = r_minus_val * np.cos(T)

# ----- Plot & animate -----
fig = plt.figure(figsize=(10,9))
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('black')

# plot stationary surfaces once
ax.plot_surface(X_e, Y_e, Z_e, color='gold', alpha=0.35, linewidth=0, antialiased=False)
ax.plot_surface(X_h_plus, Y_h_plus, Z_h_plus, color='red', alpha=0.7, linewidth=0, antialiased=False)
if r_minus_val > 0:
    ax.plot_surface(X_h_minus, Y_h_minus, Z_h_minus, color='blue', alpha=0.6, linewidth=0, antialiased=False)

# plot trajectories static preview
colors = ['cyan', 'white', 'magenta']
lines = []
points = []
for i, (x, y, z, sol) in enumerate(traj_xyz):
    # draw full trajectory as thin line (faint)
    ln, = ax.plot(x, y, z, color=colors[i], alpha=0.3, linewidth=1)
    # moving point marker
    pt, = ax.plot([x[0]], [y[0]], [z[0]], marker='o', color=colors[i], markersize=6)
    lines.append(ln)
    points.append((pt, x, y, z))

ax.set_xlim([-60,60]); ax.set_ylim([-60,60]); ax.set_zlim([-30,30])
ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
ax.set_title("Kerr null geodesics (photon trajectories) around rotating BH", color='w')

# animation function: advance markers along each trajectory
max_len = max([len(sol.y[0]) for sol,_,_ in geodesics])
def update(frame):
    azim = 0.3*frame
    ax.view_init(elev=20, azim=azim)
    for i, (sol, Lz, Q) in enumerate(geodesics):
        x = traj_xyz[i][0]; y = traj_xyz[i][1]; z = traj_xyz[i][2]
        idx = min(frame, len(x)-1)
        points[i][0].set_data([x[idx]], [y[idx]])
        points[i][0].set_3d_properties([z[idx]])
    return [p[0] for p in points]

anim = FuncAnimation(fig, update, frames=np.arange(0, max_len, 1), interval=40, blit=False)

# In Jupyter/Colab show as JS animation:
from IPython.display import HTML
HTML(anim.to_jshtml())
